In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
#from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary
import torch.nn.functional as F


In [2]:

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

#print(os.environ.get('CUDA_VISIBLE_DEVICES'))

cuda


In [3]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

In [4]:
# def apply_fisheye_distortion(image, mask, label):

#     # 이미지 크기 가져오기
#     batch, channel, height, width = image.shape
#     image = image.permute(0, 2, 3, 1)
#     mask = mask.permute(0, 1, 2)
#     images = image.split(1, dim=0)
#     masks = mask.split(1, dim=0)

#     # 카메라 매트릭스 생성
#     focal_length = width / 4
#     center_x = width / 2
#     center_y = height / 2
#     camera_matrix = np.array([[focal_length, 0, center_x],
#                               [0, focal_length, center_y],
#                               [0, 0, 1]], dtype=np.float32)

#     # 왜곡 계수 생성
#     #dist_coeffs = np.array([0, 0.5, 0, 0], dtype=np.float32)
#     dist_coeffs = np.array([0, 0.2*label, 0, 0], dtype=np.float32)

#     # 왜곡 보정
#     undistorted_images = []
#     undistorted_masks = []
#     for i in range(batch):
#         image = images[i]
#         mask = mask[i]
#         image = image.cpu().numpy()
#         mask = mask.cpu().numpy
#         undistorted_image = cv2.undistort(image.squeeze(), camera_matrix, dist_coeffs)
#         undistorted_mask = cv2.undistort(mask.squeeze(), camera_matrix, dist_coeffs)
#         undistorted_mask = np.round(mask).astype(np.uint8)
#         undistorted_mask[undistorted_mask > 12] = 12
#         undistorted_image = torch.from_numpy(undistorted_image).to(device)
#         undistorted_mask = torch.from_numpy(undistorted_mask).to(device)
#         undistorted_images.append(undistorted_image)
#         undistorted_masks.append(undistorted_mask)

#     undistorted_images = torch.stack(undistorted_images, dim=0)
#     undistorted_masks = torch.stack(undistorted_masks, dim=0)

#     undistorted_images = undistorted_images.permute(0,3,1,2)
#     undistorted_masks = undistorted_masks.permute(0,1,2)
    
#     return undistorted_images, undistorted_masks


In [5]:
import numpy as np
import cv2
import torch

def apply_fisheye_distortion(images, masks, label):
    # 이미지 크기 가져오기
    batch, channel, height, width = images.shape

    # 카메라 매트릭스 생성
    focal_length = width / 4
    center_x = width / 2
    center_y = height / 2
    camera_matrix = np.array([[focal_length, 0, center_x],
                              [0, focal_length, center_y],
                              [0, 0, 1]], dtype=np.float32)

    # 왜곡 계수 생성
    dist_coeffs = np.array([0, 0.2 * label, 0, 0], dtype=np.float32)

    # 왜곡 보정
    undistorted_images = []
    undistorted_masks = []

    for i in range(batch):
        image = images[i].permute(1, 2, 0).cpu().numpy()  # 텐서를 NumPy 배열로 변환
        mask = masks[i].cpu().numpy()
        undistorted_image = cv2.undistort(image, camera_matrix, dist_coeffs)
        undistorted_mask = cv2.undistort(mask, camera_matrix, dist_coeffs)
        undistorted_mask = np.round(undistorted_mask).astype(np.uint8)
        undistorted_mask[undistorted_mask > 12] = 12

        # 다시 텐서로 변환
        undistorted_image = torch.from_numpy(undistorted_image).permute(2, 0, 1).float().to(device)
        undistorted_mask = torch.from_numpy(undistorted_mask).long().to(device)

        undistorted_images.append(undistorted_image)
        undistorted_masks.append(undistorted_mask)

    undistorted_images = torch.stack(undistorted_images, dim=0)
    undistorted_masks = torch.stack(undistorted_masks, dim=0)

    return undistorted_images, undistorted_masks


In [6]:
# import torch

# # 이미지 데이터 (batch, channel, height, width)
# image_data = torch.randn(4, 3, 256, 256)

# # 이미지 차원 변경 (batch, height, width, channel)
# image_data_permuted = image_data.permute(0, 2, 3, 1)

# # 이미지 크기 확인
# print(image_data_permuted.shape)

# # 4개의 이미지로 나누기
# images = image_data_permuted.split(1, dim=0)
# # 또는 images = torch.split(image_data_permuted, 1, dim=0)
# undistorted_images = []
# # 4개 이미지의 크기 확인
# for i, image in enumerate(images):
#     print(f"Image {i + 1} shape: {image.shape}")
#     undistorted_images.append(images[i].squeeze())

# undistorted_images2 = torch.stack(undistorted_images, dim=0)
# undistorted_images3 = undistorted_images2.permute(0,3,1,2)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        directory_path = './data/224'
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path)
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = np.round(mask).astype(np.uint8)
        mask[mask > 12] = 12 #배경을 픽셀값 12로 간주
        mask += 1
        mask[mask == 13] = 0

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

# class CustomDataset_target(Dataset):
#     def __init__(self, csv_file, transform=None, infer=False):
#         self.data = pd.read_csv(csv_file)
#         self.transform = transform
#         self.infer = infer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         directory_path = "/mnt/nas27/Dataset/Samsung_DM"
#         img_path = self.data.iloc[idx, 1]
#         img_path = os.path.join(directory_path, img_path[2:])
#         image = cv2.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#         if self.infer:
#             if self.transform:
#                 image = self.transform(image=image)['image']
#             return image


#         if self.transform:
#             augmented = self.transform(image=image)
#             image = augmented['image']
            

#         return image
     

transform = A.Compose(
    [   
        #A.Resize(224, 224),
        #A.Resize(128, 128),
        A.Normalize(),
        
        # 변형
        # A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        # A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

In [8]:
#Unet의 기본이 되는 conv블럭
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)  # 여기서 in_channels는 out_channels와 동일해야 합니다.
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        return x

#인코더 블럭
class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock,self).__init__()
        self.convblock1 = ConvBlock(in_channels, out_channels)  # 첫 번째 ConvBlock의 in_channels는 입력 이미지의 채널 수와 일치해야 합니다.
        #self.convblock2 = ConvBlock(out_channels, out_channels)  # 두 번째 ConvBlock의 in_channels는 out_channels와 일치해야 합니다.
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self,x):
        x = self.convblock1(x)
        #x = self.convblock2(x)
        p = self.maxpool(x)
        return x , p
#디코더 블럭
#디코더는 업샘플링 이후 스킵연결과 붙어서 convblock을 통과해야함
#skip보다 작은 x x먼저 업샘플링 32 -> 64 , skip과 결합 6464 
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock,self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1)#x 업샘플링
        self.convblock1 = ConvBlock(channels*2, channels)#차원감소
        #self.convblock2 = ConvBlock(channels, channels)
    def forward(self,x,skip):
        x = self.upsample(x)
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        #x = self.convblock2(x)
        return x

###########################################
class GradReverse(torch.autograd.Function):
    def forward(self, x):
        return x.view_as(x)

    def backward(self, grad_output): # 역전파 시에 gradient에 음수를 취함
        return (grad_output * -1)

class domain_classifier(nn.Module):
    def __init__(self):
        super(domain_classifier, self).__init__()
        self.fc1 = nn.Linear(224*224*64, 10)
        self.fc2 = nn.Linear(10, 4) # source = 0, target = 1 회귀 가정

    def forward(self, x):
        x = x.view(-1, 224*224*64)
        x = GradReverse.apply(x) # gradient reverse
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

###########################################


#Unet구조 middle의 xm값의 움직임에 주의
class Unet(nn.Module):
    def __init__(self,n_classes):
        super(Unet,self).__init__()
        self.encoder1 = EncoderBlock(3,64)
        self.encoder2 = EncoderBlock(64,128)
        self.encoder3 = EncoderBlock(128,256)
        self.encoder4 = EncoderBlock(256,512)
        
        self.middleconv = ConvBlock(512,1024)
        
        
        self.decoder4 = DecoderBlock(512)
        self.decoder3 = DecoderBlock(256)
        self.decoder2 = DecoderBlock(128)
        self.decoder1 = DecoderBlock(64)
        self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        
        self.domain_classifier = domain_classifier()
                                        

    def forward(self,x):
        x1,p = self.encoder1(x)#3->64   #P:256,256 x1 :512,512
        x2,p = self.encoder2(p)#64->128 #P:128,128 x2:256,256
        x3,p = self.encoder3(p)#128->256#p:64,64 x3:128,128
        x4,p = self.encoder4(p)#256->512#p:32,32 x4:64,64
        
        xm = self.middleconv(p)#512->1024#32,32
        
        x = self.decoder4(xm,x4)#뉴런:1024->512->512 #출력tensor:64,64
        x = self.decoder3(x,x3)#뉴런:512->256->256 #출력tensor:128,128
        x = self.decoder2(x,x2)#뉴런:256->128->128 #출력tensor:256,256
        x = self.decoder1(x,x1)#뉴런:128->64->64 #출력tensor:512,512

        x_c = self.segmap(x)
        x_d = self.domain_classifier(x)
        # print("x_c", x_c.shape)
        # print("x_d", x_d.shape)
        return x_c, x_d

In [9]:
# import pickle

# # 저장된 class_weights를 불러옵니다.
# class_weights_path = 'CLASS_WEIGHTS.pkl'

# with open(class_weights_path, 'rb') as file:
#     CLASS_WEIGHTS = pickle.load(file)

# print(CLASS_WEIGHTS)

## Loss Function

In [10]:
# loss function과 optimizer 정의

class DANN_Loss(nn.Module):
    def __init__(self):
        super(DANN_Loss, self).__init__()

        #self.CE = nn.CrossEntropyLoss(weight=CLASS_WEIGHTS) # 0~9 class 분류용
        self.CE = nn.CrossEntropyLoss()
        #self.BCE = nn.CrossEntropyLoss() # 도메인 분류용
        
    # result : DANN_CNN에서 반환된 값
    # label : 숫자 0 ~ 9에 대한 라벨
    # domain_num : 0(source) or 1(target)
    def forward(self, result, label, domain_num, alpha = 1):
        label_logits, domain_logits = result # DANN_CNN의 결과

        batch_size = domain_logits.shape[0]

        #domain_target = torch.FloatTensor([domain_num] * batch_size).unsqueeze(1).to(device)
        domain_target = torch.LongTensor([domain_num] * batch_size).to(device)
        # print("segment_mask : ", label.shape)
        # print("domain_answer : ", domain_target.shape)
        segment_loss = self.CE(label_logits, label) # class 분류 loss
        domain_loss = self.CE(domain_logits, domain_target) # domain 분류 loss
        
        loss = segment_loss + alpha * domain_loss

        return loss
    

loss_fn = DANN_Loss().to(device)

#criterion =nn.CrossEntropyLoss()
#domain_criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss(weight=class_weights)

In [11]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [12]:
LR = 0.001
EP = 10
BATCH_SIZE = 8
ACCMULATION_STEP = 1 
N_CLASSES = 13 #IoU 점수측정하기 위한 클래스의 개수
ALPHA = 0.5
# model 초기화
#model = Unet_resnet18(n_classes = N_CLASSES).to(device)
#model = ResNet50(num_classes=N_CLASSES).to(device)
model = Unet(n_classes = N_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

optimizer.zero_grad() 
#########
# source_dataset = CustomDataset(csv_file='./data/DA_csv/train_source.csv', transform=transform)
# source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True)
# target_dataset = CustomDataset(csv_file='./data/DA_csv/fish_train_source.csv', transform=transform)
# target_dataloader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=True)

# val_source_dataset = CustomDataset(csv_file='./data/DA_csv/val_source.csv', transform=transform)
# val_source_dataloader = DataLoader(val_source_dataset, batch_size=BATCH_SIZE, shuffle=False)
# val_target_dataset = CustomDataset(csv_file='./data/DA_csv/fish_val_source.csv', transform=transform)
# val_target_dataloader = DataLoader(val_target_dataset, batch_size=BATCH_SIZE, shuffle=False)
##########
source_dataset = CustomDataset(csv_file='./data/train_source.csv', transform=transform)
source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_target_dataset = CustomDataset(csv_file='./data/DA_csv/fish_val_source.csv', transform=transform)
val_target_dataloader = DataLoader(val_target_dataset, batch_size=BATCH_SIZE, shuffle=False)

# source_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)#
# source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
# s_valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
# s_valid_dataloader = DataLoader(s_valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
# #target_dataset = CustomDataset_target(csv_file='./f_train_source.csv', transform=transform)
# target_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./fish_val_source.csv'), transform=transform)
# target_dataloader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

# t_test_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./test.csv'), transform=transform, infer=True)
# t_test_dataloader = DataLoader(t_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

In [13]:
# import pandas as pd
# import os
# import cv2
# import matplotlib.pyplot as plt
# a1 = pd.read_csv("./data/6_fish_source.csv")
# p = "./data/224/"
# a2 = os.path.join(p, a1.iloc[3,2])
# a3 = cv2.imread(a2)
# a4 = cv2.cvtColor(a3, cv2.COLOR_BGR2GRAY)
# a4 = np.round(a4).astype(np.uint8)
# a5 = a4*20

# plt.imshow(a5, cmap='gray')
# plt.axis('off')
# plt.show()

In [14]:
import random
#torch.cuda.empty_cache()
# import wandb


# wandb.init(
#     # set the wandb project where this run will be logged
#     project="practice_10_19",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR,
#     "architecture": "CNN",
#     "dataset": "Samsung",
#     "epochs": EP,
#     }
# )

for epoch in range(EP):
    # 클래스별 IoU를 누적할 리스트 초기화
    train_class_ious = []
    fish_train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0

    for source_images, source_masks in tqdm(source_dataloader):
        label = random.randint(0,3)
        source_images, source_masks = apply_fisheye_distortion(source_images, source_masks, label)
        source_images = source_images.float().to(device)
        source_masks = source_masks.long().to(device)

        optimizer.zero_grad()
        source_outputs = model(source_images)

        source_loss = loss_fn(source_outputs, source_masks, label, alpha = ALPHA)

        loss = source_loss
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()
        #scheduler.step()
                # train 클래스별 IoU 계산
        source_outputs = torch.softmax(source_outputs[0], dim=1).cpu()
        source_outputs = torch.argmax(source_outputs, dim=1).numpy()

        for class_id in range(N_CLASSES):
            iou = calculate_iou_per_class(np.array(source_masks.cpu()), np.array(source_outputs), class_id)
            train_class_ious.append(iou)

    train_class_ious = np.array(train_class_ious).reshape(N_CLASSES, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    print("--IoU Scores Train--")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    train_mIoU = np.mean(train_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {(epoch_loss/len(source_dataloader))}")
    print(f"Train mIoU: {train_mIoU}" )
    print("___________________________________________________________________________________________\n")


    ################################################################
    # 클래스별 IoU를 누적할 리스트 초기화
    val_class_ious = []
    fish_val_class_ious = []
    epoch_loss = 0
    # 학습
    with torch.no_grad():
        model.eval()

        for target_images, target_masks in tqdm(val_target_dataloader):

            target_images = target_images.float().to(device)
            target_masks = target_masks.long().to(device)

            target_outputs = model(target_images)

            target_loss = loss_fn(target_outputs, target_masks, 0, alpha = ALPHA)

            loss = target_loss

            epoch_loss += loss.item()

            # train 클래스별 IoU 계산
            target_outputs = torch.softmax(target_outputs[0], dim=1).cpu()
            target_outputs = torch.argmax(target_outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(target_masks.cpu()), np.array(target_outputs), class_id)
                fish_val_class_ious.append(iou)

    fish_val_class_ious = np.array(fish_val_class_ious).reshape(N_CLASSES, -1)
    fish_val_class_ious = np.mean(fish_val_class_ious, axis=1)
    print()
    print("--IoU Scores Fish val--")
    for class_id, iou in enumerate(fish_val_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    fish_val_mIoU = np.mean(fish_val_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Valid Loss: {(epoch_loss/len(val_target_dataloader))}")
    print(f"Valid mIoU: {fish_val_mIoU}" )
    print("___________________________________________________________________________________________\n")

#     # log metrics to wandb
#     wandb.log({"train score": train_mIoU})
#     wandb.log({"val score": fish_val_mIoU})
    
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()


  0%|          | 0/275 [00:00<?, ?it/s]

 52%|█████▏    | 142/275 [03:36<03:23,  1.53s/it]


KeyboardInterrupt: 